In [33]:
import glob
import os

import pandas as pd
import numpy as np
from numpy import zeros, newaxis
import cv2

import matplotlib.pyplot as plt
import seaborn

In [34]:
abnormal_path_train =  'MRNet-v1.0/train-abnormal.csv'
acl_path_train = 'MRNet-v1.0/train-acl.csv'
meniscus_train =  'MRNet-v1.0/train-meniscus.csv'


abnormal_path_valid =  'MRNet-v1.0/valid-abnormal.csv'
acl_path_valid = 'MRNet-v1.0/valid-acl.csv'
meniscus_valid =  'MRNet-v1.0/valid-meniscus.csv'

In [35]:
def load_labels(abnormal_path, acl_path, meniscus_path):

    df_labels_abnormal =  pd.read_csv(abnormal_path, names=['Image_Name', 'Abnormal_Label'])
    df_labels_acl = pd.read_csv(acl_path, names=['Image_Name', 'Acl_Label'])
    df_labels_meniscus = pd.read_csv(meniscus_path, names=['Image_Name', 'Meniscus_Label'])

    df_lables_combined = pd.merge(df_labels_abnormal, df_labels_acl, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined = pd.merge(df_lables_combined, df_labels_meniscus, how = 'left', left_on=['Image_Name'], right_on=['Image_Name'])
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].astype(str)
    df_lables_combined['Image_Name'] = df_lables_combined['Image_Name'].apply(lambda x: x.zfill(4))

    return df_lables_combined

In [36]:
df_labels_train = load_labels(abnormal_path_train, acl_path_train, meniscus_train)
df_labels_train

df_labels_valid = load_labels(abnormal_path_valid, acl_path_valid, meniscus_valid)
df_labels_valid

,Image_Name,Abnormal_Label,Acl_Label,Meniscus_Label
0,1130,0,0,0
1,1131,0,0,0
2,1132,0,0,0
3,1133,0,0,0
4,1134,0,0,0
...,...,...,...,...
115,1245,1,1,1
116,1246,1,1,1
117,1247,1,0,1
118,1248,1,1,1


In [37]:
glob.glob('MRNet-v1.0/*')


['MRNet-v1.0\\train',
 'MRNet-v1.0\\train-abnormal.csv',
 'MRNet-v1.0\\train-acl.csv',
 'MRNet-v1.0\\train-meniscus.csv',
 'MRNet-v1.0\\valid',
 'MRNet-v1.0\\valid-abnormal.csv',
 'MRNet-v1.0\\valid-acl.csv',
 'MRNet-v1.0\\valid-meniscus.csv']

In [38]:
# Read Traning Data
def read_data(train_path, df_labels):
    axial_images = []
    coronal_images = []
    sagittal_images = []

    axial_labels = []
    coronal_labels = []
    sagittal_labels = []

    # train_path = '/kaggle/input/mrnet-v1/MRNet-v1.0/train'
    train_folders = glob.glob(train_path + '/*')

    for folder in train_folders:
        train_data = glob.glob(folder + '/*.npy')
        print('Total samples in data:', len(train_data))
        print('Loading Data From :   ', folder)

        for i in train_data:
            #             print(i)
            img_array = np.load(i)
            start, end = (img_array.shape[0]/2)-8, (img_array.shape[0]/2)+8
            image_tensor = img_array[int(start):int(end),:,:].reshape(256,256,16)
            if 'axial' in i:
                axial_images.append(image_tensor)
                image_name = i.rsplit('/')[-1].split('.')[0]
                axial_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'coronal' in i:
                coronal_images.append(image_tensor)
                image_name = i.rsplit('/')[-1].split('.')[0]
                coronal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])
            if 'sagittal' in i:
                sagittal_images.append(image_tensor)
                image_name = i.rsplit('/')[-1].split('.')[0]
                sagittal_labels.append(df_labels[df_labels['Image_Name']==image_name].iloc[:, 1:].values[0])

        print('Loading Complete From :   ', folder)
        print('\n\n')
    return axial_images, coronal_images, sagittal_images, axial_labels

train_path =  'MRNet-v1.0/train'
valid_path =  'MRNet-v1.0/valid'

# Load Data Here.
axial_images_train, coronal_images_train, sagittal_images_train, axial_labels_train = read_data(train_path, df_labels_train)

Total samples in data: 1130
Loading Data From :    MRNet-v1.0/train\axial


IndexError: index 0 is out of bounds for axis 0 with size 0

Total samples in data: 1130
Loading Data From :    MRNet-v1.0/train\axial


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.layers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [ ]:
# Two Input Model

def model ():
    visible1 = Input(shape=(256,256,16))
    
    conv11 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(visible1)
    pool11 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv11)
    
    conv12 = Conv2D( filters=16, kernel_size=(5,5), activation="relu")(pool11)
    pool12 = MaxPool2D(pool_size=(2,2),strides=(2,2))(conv12)
    
    conv13 = Conv2D(filters=16, kernel_size=(5,5), activation='relu')(pool12)
    pool13 = MaxPooling2D(pool_size=(2, 2),strides=(2,2))(conv13)
    flat1 = Flatten()(pool13)
    
    return flat1, visible1
model()

# merge input models
model_1, Visible_1 = model()
model_2, Visible_2 = model()

merge = concatenate([model_1, model_2])

hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(5, activation='relu')(hidden1)
output = Dense(3, activation='sigmoid')(hidden2)
model = Model(inputs=[Visible_1, Visible_2], outputs=output)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 input_3 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                6)]                                                               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 252, 252, 16  6416        ['input_2[0][0]']                
                                )                                                             

In [ ]:
model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.SGD( learning_rate = 0.001, momentum = 0.9),metrics=["accuracy"])

# Give Class Weights to Balance Results
class_weight = {0: 1.,
                1: 50.,
                2: 50.}

model.fit(x = [np.array(coronal_images_train), np.array(sagittal_images_train)],
          y = np.array(axial_labels_train),
          epochs=5,
          batch_size=32,
          validation_split=0.20,
          class_weight=class_weight)

# model.save_weights('model_3_Input.h5')

NameError: name 'coronal_images_train' is not defined